In [1]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 38.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 74.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 82.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.2 MB/s eta 0:00:0

In [2]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub

from keras.utils import to_categorical
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, Flatten, Dense, Dropout
from keras.models import Sequential

from sklearn.decomposition import PCA

from sklearn.neural_network import MLPRegressor

import numpy as np

import xgboost as xgb

In [3]:
url = "https://raw.githubusercontent.com/Tejanikhil/Shared-ASAG/main"
df = pd.read_csv(url + "/SemEvalData.csv")

In [4]:
import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords

# Download the stopwords corpus if not already downloaded
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
def extract_pos(text):
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    pos_words = [word for word, pos in pos_tags if pos.startswith('VB') or pos.startswith('JJ') or pos.startswith('NN')]
    return ' '.join(pos_words)

def process_dataset(text):
    lower_cased = text.lower()
    tokens = nltk.word_tokenize(lower_cased)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    preprocessed = ' '.join(filtered_tokens)
    return preprocessed

In [6]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(df.loc[:,["Actual Answer","Student Answer"]], df["Grade"], test_size=0.1, random_state=42)

In [7]:
TRAIN = train_X
TRAIN["LABELS"] = train_y
TEST = test_X
TEST["LABELS"] = test_y

In [8]:
train_X["Actual Answer"] = train_X["Actual Answer"].apply(extract_pos).apply(process_dataset)
train_X["Student Answer"] = train_X["Student Answer"].apply(extract_pos).apply(process_dataset)

test_X["Actual Answer"] = test_X["Actual Answer"].apply(extract_pos).apply(process_dataset)
test_X["Student Answer"] = test_X["Student Answer"].apply(extract_pos).apply(process_dataset)

In [9]:
print(TRAIN.shape)
TRAIN = TRAIN.replace("", pd.NA).dropna()
print(TRAIN.shape)
TEST = TEST.replace("", pd.NA).dropna()

(2197, 3)
(2184, 3)


In [10]:
train_X = TRAIN.drop("LABELS", axis = 1)
train_y = TRAIN.iloc[:,-1]

In [11]:
sentences = list(train_X["Actual Answer"]) + list(train_X["Student Answer"])

In [12]:
rm -rf ~/.cache/huggingface/transformers/bert-base-uncased

In [13]:
from flair.embeddings import FlairEmbeddings
from flair.data import Sentence
import flair

In [14]:
flair_forward = flair.embeddings.FlairEmbeddings('multi-forward')
flair_backward = flair.embeddings.FlairEmbeddings('multi-backward')
flair_en_forward = flair.embeddings.FlairEmbeddings('en-forward')

2023-06-08 16:33:58,568 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/lm-jw300-forward-v0.1.pt not found in cache, downloading to /tmp/tmpkbxbha96


100%|██████████| 165M/165M [00:08<00:00, 20.5MB/s]

2023-06-08 16:34:07,446 copying /tmp/tmpkbxbha96 to cache at /root/.flair/embeddings/lm-jw300-forward-v0.1.pt


2023-06-08 16:34:07,935 removing temp file /tmp/tmpkbxbha96
2023-06-08 16:34:19,265 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/lm-jw300-backward-v0.1.pt not found in cache, downloading to /tmp/tmpxea3sr4r


100%|██████████| 165M/165M [00:08<00:00, 20.7MB/s]

2023-06-08 16:34:27,997 copying /tmp/tmpxea3sr4r to cache at /root/.flair/embeddings/lm-jw300-backward-v0.1.pt


2023-06-08 16:34:28,320 removing temp file /tmp/tmpxea3sr4r
2023-06-08 16:34:29,305 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/news-forward-0.4.1.pt not found in cache, downloading to /tmp/tmpnlknn5wi


100%|██████████| 69.7M/69.7M [00:04<00:00, 18.0MB/s]

2023-06-08 16:34:33,762 copying /tmp/tmpnlknn5wi to cache at /root/.flair/embeddings/news-forward-0.4.1.pt


2023-06-08 16:34:33,851 removing temp file /tmp/tmpnlknn5wi


In [15]:
flair_news_forward = flair.embeddings.FlairEmbeddings('news-forward')
flair_news_backward = flair.embeddings.FlairEmbeddings('news-backward')

2023-06-08 16:34:34,707 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/news-backward-0.4.1.pt not found in cache, downloading to /tmp/tmpvpfahwtx


100%|██████████| 69.7M/69.7M [00:04<00:00, 18.2MB/s]

2023-06-08 16:34:39,126 copying /tmp/tmpvpfahwtx to cache at /root/.flair/embeddings/news-backward-0.4.1.pt


2023-06-08 16:34:39,247 removing temp file /tmp/tmpvpfahwtx


In [103]:
train_Embeddings_df = pd.DataFrame({"Student_Answer_Embeddings":train_X["Student Answer"], "Reference_Answer_Embeddings":train_X["Actual Answer"]})
test_Embeddings_df = pd.DataFrame({"Student_Answer_Embeddings":test_X["Student Answer"], "Reference_Answer_Embeddings":test_X["Actual Answer"]})

In [104]:
def get_flair_multi_forward_embeddings(text):
  sentence = Sentence(text)
  flair_forward.embed(sentence)
  embeds = [np.array(i.embedding.cpu()) for i in sentence]
  return np.mean(np.array(embeds),axis = 0)

In [105]:
def get_flair_multi_backward_embeddings(text):
  sentence = Sentence(text)
  flair_backward.embed(sentence)
  embeds = [np.array(i.embedding.cpu()) for i in sentence]
  return np.mean(np.array(embeds),axis = 0)

In [106]:
def get_flair_news_forward_embeddings(text):
  sentence = Sentence(text)
  flair_news_forward.embed(sentence)
  embeds = [np.array(i.embedding.cpu()) for i in sentence]
  return np.mean(np.array(embeds),axis = 0)

In [107]:
def get_flair_news_backward_embeddings(text):
  sentence = Sentence(text)
  flair_news_backward.embed(sentence)
  embeds = [np.array(i.embedding.cpu()) for i in sentence]
  return np.mean(np.array(embeds),axis = 0)

In [108]:
def get_flair_en_forward_embeddings(text):
  sentence = Sentence(text)
  flair_en_forward.embed(sentence)
  embeds = [np.array(i.embedding.cpu()) for i in sentence]
  return np.mean(np.array(embeds),axis = 0)

In [109]:
get_flair_en_forward_embeddings("I am a good boy")

array([ 0.00305832, -0.00205429,  0.0675773 , ..., -0.00094282,
       -0.02816246,  0.0060261 ], dtype=float32)

In [110]:
train_Embeddings_df["Student_Answer_Embeddings"] = train_Embeddings_df["Student_Answer_Embeddings"].apply(get_flair_multi_backward_embeddings)
train_Embeddings_df["Reference_Answer_Embeddings"] = train_Embeddings_df["Reference_Answer_Embeddings"].apply(get_flair_multi_backward_embeddings)

test_Embeddings_df["Student_Answer_Embeddings"] = test_Embeddings_df["Student_Answer_Embeddings"].apply(get_flair_multi_backward_embeddings)
test_Embeddings_df["Reference_Answer_Embeddings"] = test_Embeddings_df["Reference_Answer_Embeddings"].apply(get_flair_multi_backward_embeddings)

In [111]:
train_Embeddings_df

,Student_Answer_Embeddings,Reference_Answer_Embeddings
199,"[-0.044092108, 2.7796827e-05, -0.005076336, 1....","[0.003215659, 0.00045086298, -0.009349719, -0...."
229,"[0.04664183, 0.00029912023, -0.019464111, -0.0...","[0.007407506, 0.0013505139, -0.00821017, -0.00..."
521,"[0.014759156, -0.00025871416, -0.016955964, -0...","[0.0098342, 0.00046607698, -0.016177842, -0.00..."
651,"[0.007367341, 0.0014515653, -0.02205511, -0.00...","[0.016333904, 7.0973088e-06, -0.0071644085, 0...."
450,"[0.0020619135, 0.00034114835, -0.010512607, -0...","[0.0055840593, 0.0020162372, -0.012744251, -0...."
...,...,...
1638,"[0.0034005782, 0.00046609214, -0.051265948, 0....","[0.0052932478, 0.0002658487, -0.004874393, -0...."
1095,"[0.013816073, 0.00052707666, -0.0077710063, -0...","[0.020783672, 0.00011875443, -0.01655055, -0.0..."
1130,"[0.017660877, 0.00033991222, -0.030030642, -0....","[0.016915947, 0.002964161, -0.041464966, -0.00..."
1294,"[0.0033517703, 0.00238781, -0.019812778, -0.00...","[0.0009521609, 0.0012425041, -0.003282945, -0...."


In [112]:
train_X["Subtracted_Embeddings"] = train_Embeddings_df.apply(lambda x:x["Student_Answer_Embeddings"]-x["Reference_Answer_Embeddings"], axis = 1)
test_X["Subtracted_Embeddings"] = test_Embeddings_df.apply(lambda x:x["Student_Answer_Embeddings"]-x["Reference_Answer_Embeddings"], axis = 1)

In [113]:
train_df = pd.DataFrame(list(train_X["Subtracted_Embeddings"]))
test_df = pd.DataFrame(list(test_X["Subtracted_Embeddings"]))

In [114]:
train_df.to_csv("flair_en_forward_embeddings.csv")

In [115]:
pca = PCA(n_components=100)  # Specify the number of components you want to keep
train_df_pca = pca.fit_transform(train_df)

In [116]:
test_df_pca = pca.transform(test_df)

In [117]:
ncols = train_df_pca.shape[1]

In [118]:
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(ncols,1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(train_df_pca, train_y, epochs=10, batch_size=32, verbose=1)

y_true = test_y
y_preds = model.predict(test_df_pca)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
cnn_mae = mean_absolute_error(y_true, np.round(y_preds, 1))
cnn_mse = mean_squared_error(y_true, np.round(y_preds, 1))
cnn_rmse = mean_squared_error(y_true, np.round(y_preds, 1), squared=False)
cnn_mape = mean_absolute_percentage_error(y_true, y_preds, multioutput = "uniform_average")
cnn_r2 = r2_score(y_true, y_preds)

print("Mean Absolute Error (MAE): ", np.round(cnn_mae, 2))
print("Mean Squared Error (MSE): ", np.round(cnn_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(cnn_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(cnn_mape, 2))
print("R2 Score: ", np.round(cnn_r2, 2))

Epoch 1/10
69/69 [==============================] - 1s 3ms/step - loss: 4.8366
Epoch 2/10
69/69 [==============================] - 0s 3ms/step - loss: 2.9408
Epoch 3/10
69/69 [==============================] - 0s 3ms/step - loss: 2.7553
Epoch 4/10
69/69 [==============================] - 0s 3ms/step - loss: 2.6501
Epoch 5/10
69/69 [==============================] - 0s 3ms/step - loss: 2.5911
Epoch 6/10
69/69 [==============================] - 0s 3ms/step - loss: 2.5670
Epoch 7/10
69/69 [==============================] - 0s 3ms/step - loss: 2.5264
Epoch 8/10
69/69 [==============================] - 0s 3ms/step - loss: 2.5044
Epoch 9/10
69/69 [==============================] - 0s 3ms/step - loss: 2.5097
Epoch 10/10
8/8 [==============================] - 0s 5ms/step
Mean Absolute Error (MAE):  1.27
Mean Squared Error (MSE):  2.12
Root Mean Squared Error (RMSE):  1.46
Mean Absolute Percentage Error (MAPE):  505646149148505.1
R2 Score:  -0.08


In [119]:
# Create an MLP regressor with a single linear output layer
mlp = MLPRegressor(hidden_layer_sizes=(100,50,25,1), activation='identity', solver='lbfgs')

# Train the MLP regressor
mlp.fit(train_df_pca, train_y)

y_preds = mlp.predict(test_df_pca)

y_true = test_y

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
mlp_mae = mean_absolute_error(y_true, np.round(y_preds, 1))
mlp_mse = mean_squared_error(y_true, np.round(y_preds, 1))
mlp_rmse = mean_squared_error(y_true, np.round(y_preds, 1), squared=False)
mlp_mape = mean_absolute_percentage_error(y_true, y_preds, multioutput = "uniform_average")
mlp_r2 = r2_score(y_true, y_preds)

print("Mean Absolute Error (MAE): ", np.round(mlp_mae, 2))
print("Mean Squared Error (MSE): ", np.round(mlp_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(mlp_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(mlp_mape, 2))
print("R2 Score: ", np.round(mlp_r2, 2))

Mean Absolute Error (MAE):  1.11
Mean Squared Error (MSE):  1.86
Root Mean Squared Error (RMSE):  1.37
Mean Absolute Percentage Error (MAPE):  499653511638660.2
R2 Score:  0.06


In [120]:
from sklearn.linear_model import LinearRegression
LR_model = LinearRegression()
LR_model.fit(train_df_pca, train_y)

preds = LR_model.predict(test_df_pca)
y_preds = [round(y_val, 1) for y_val in preds]
actuals = test_y
y_true = [round(y_val, 1) for y_val in actuals]

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
LR_mae = mean_absolute_error(y_true, np.round(y_preds, 1))
LR_mse = mean_squared_error(y_true, np.round(y_preds, 1))
LR_rmse = mean_squared_error(y_true, np.round(y_preds, 1), squared=False)
LR_mape = mean_absolute_percentage_error(y_true, y_preds, multioutput = "uniform_average")
LR_r2 = r2_score(y_true, y_preds)

print("Mean Absolute Error (MAE): ", np.round(LR_mae, 2))
print("Mean Squared Error (MSE): ", np.round(LR_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(LR_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(LR_mape, 2))
print("R2 Score: ", np.round(LR_r2, 2))

Mean Absolute Error (MAE):  1.12
Mean Squared Error (MSE):  1.9
Root Mean Squared Error (RMSE):  1.38
Mean Absolute Percentage Error (MAPE):  499991471902436.2
R2 Score:  0.06


In [121]:
from sklearn import neighbors
rmse_val = []
mae_val = []
mse_val = []
r2_val = []
for K in range(2,20):
    knn_model = neighbors.KNeighborsRegressor(n_neighbors = K)
    knn_model.fit(train_df_pca, train_y)  #fit the model
    pred=knn_model.predict(test_df_pca) #make prediction on test set
    
    mae = mean_absolute_error(y_true, np.round(pred, 1))
    mae_val.append(mae)
    
    mse = mean_squared_error(y_true, np.round(pred, 1))
    mse_val.append(mse)
    
    rmse = mean_squared_error(y_true, np.round(pred, 1), squared=False)
    rmse_val.append(rmse)
    
    r2 = r2_score(y_true, pred)
    r2_val.append(r2)

y_true = test_y
knn_preds = knn_model.predict(test_df_pca)

min_val = min(r2_val)
index = r2_val.index(min_val)
knn_r2 = min_val
knn_rmse = rmse_val[index]
knn_mae = mae_val[index]
knn_mse = mse_val[index]

print("Mean Absolute Error (MAE): ", np.round(knn_mae, 2))
print("Mean Squared Error (MSE): ", np.round(knn_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(knn_rmse, 2))
print("R2 Score: ", np.round(knn_r2, 2))

Mean Absolute Error (MAE):  1.05
Mean Squared Error (MSE):  2.49
Root Mean Squared Error (RMSE):  1.58
R2 Score:  -0.24


In [122]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
ada_reg = AdaBoostRegressor(n_estimators = 100, learning_rate=0.5)

ada_reg.fit(train_df, train_y)

ada_reg_pred = ada_reg.predict(pd.DataFrame(test_df))
ada_reg_mae = mean_absolute_error(test_y, np.round(ada_reg_pred, 1))
ada_reg_mse = mean_squared_error(test_y, np.round(ada_reg_pred, 1))
ada_reg_mape = mean_absolute_percentage_error(test_y, np.round(ada_reg_pred,1), multioutput = "uniform_average")
ada_reg_rmse = mean_squared_error(test_y, np.round(ada_reg_pred, 1), squared=False)
ada_reg_r2 = r2_score(test_y, ada_reg_pred)

print("Mean Absolute Error (MAE): ", np.round(ada_reg_mae, 2))
print("Mean Squared Error (MSE): ", np.round(ada_reg_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(ada_reg_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(ada_reg_mape, 2))
print("R2 Score: ", np.round(ada_reg_r2, 2))

Mean Absolute Error (MAE):  1.18
Mean Squared Error (MSE):  1.87
Root Mean Squared Error (RMSE):  1.37
Mean Absolute Percentage Error (MAPE):  476094817750595.6
R2 Score:  0.05


In [123]:
gb_reg = GradientBoostingRegressor(n_estimators = 50, max_depth=5)

gb_reg.fit(train_df_pca, train_y)

gb_reg_pred = gb_reg.predict(pd.DataFrame(test_df_pca))
gb_reg_mae = mean_absolute_error(test_y, np.round(gb_reg_pred, 1))
gb_reg_mse = mean_squared_error(test_y, np.round(gb_reg_pred, 1))
gb_reg_mape = mean_absolute_percentage_error(test_y, np.round(gb_reg_pred,1), multioutput = "uniform_average")
gb_reg_rmse = mean_squared_error(test_y, np.round(gb_reg_pred, 1), squared=False)
gb_reg_r2 = r2_score(test_y, gb_reg_pred)

print("Mean Absolute Error (MAE): ", np.round(gb_reg_mae, 2))
print("Mean Squared Error (MSE): ", np.round(gb_reg_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(gb_reg_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(gb_reg_mape, 2))
print("R2 Score: ", np.round(gb_reg_r2, 2))

Mean Absolute Error (MAE):  1.1
Mean Squared Error (MSE):  1.86
Root Mean Squared Error (RMSE):  1.36
Mean Absolute Percentage Error (MAPE):  553299382791232.56
R2 Score:  0.06


In [124]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', seed = 123)

xg_reg.fit(train_df_pca, train_y)

# print("Best parameters: ", xg_boost_grid_search.best_params_)
# print("Best score: ", xg_boost_grid_search.best_score_)
xg_reg_pred = xg_reg.predict(pd.DataFrame(test_df_pca))
xg_reg_mae = mean_absolute_error(test_y, np.round(xg_reg_pred, 1))
xg_reg_mse = mean_squared_error(test_y, np.round(xg_reg_pred, 1))
xg_reg_mape = mean_absolute_percentage_error(test_y, np.round(xg_reg_pred,1), multioutput = "uniform_average")
xg_reg_rmse = mean_squared_error(test_y, np.round(xg_reg_pred, 1), squared=False)
xg_reg_r2 = r2_score(test_y, xg_reg_pred)

print("Mean Absolute Error (MAE): ", np.round(xg_reg_mae, 2))
print("Mean Squared Error (MSE): ", np.round(xg_reg_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(xg_reg_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(xg_reg_mape, 2))
print("R2 Score: ", np.round(xg_reg_r2, 2))

Mean Absolute Error (MAE):  1.1
Mean Squared Error (MSE):  2.15
Root Mean Squared Error (RMSE):  1.47
Mean Absolute Percentage Error (MAPE):  562490399864138.5
R2 Score:  -0.09
